## Query Database

In [5]:
import face_recognition
import cv2
import sqlite3
import os

DATABASE_NAME = "face_data.db"
KNOWN_FACES_DIR = "known_faces"

def create_database():
    # Connect to the database or create it if it doesn't exist
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()

    # Create a table to store face data if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS face_data (
            id INTEGER PRIMARY KEY,
            email TEXT UNIQUE,
            name TEXT UNIQUE,
            age INTEGER,
            occupation TEXT,
            face_encoding TEXT,
            image_path TEXT
        )
    ''')

    # Save the changes and close the connection
    conn.commit()
    conn.close()

def encode_image(image_path):
    image = face_recognition.load_image_file(image_path)
    face_encodings = face_recognition.face_encodings(image)

    if len(face_encodings) > 0:
        return face_encodings[0]  # Only encode the first face found in the image
    else:
        return None

def check_existing_data(email, name, face_encoding, image_path):
    try:
        # Connect to the database
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Check if the email, name, face encoding, or image path already exists in the database
        cursor.execute('''
            SELECT * FROM face_data WHERE email=? OR name=? OR face_encoding=? OR image_path=?
        ''', (email, name, face_encoding, image_path))

        existing_data = cursor.fetchone()

        # Close the connection
        conn.close()

        return existing_data

    except sqlite3.Error as e:
        print("Error while checking existing data:", e)
        return None

def insert_data(email, name, age, occupation, face_encoding, image_path):
    try:
        # Check if the data already exists
        existing_data = check_existing_data(email, name, face_encoding, image_path)
        if existing_data:
            print("Data already exists in the database")
            return

        # Connect to the database
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Insert data into the table
        cursor.execute('''
            INSERT INTO face_data (email, name, age, occupation, face_encoding, image_path)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (email, name, age, occupation, face_encoding, image_path))

        # Save the changes and close the connection
        conn.commit()
        conn.close()

        print("Data inserted successfully!")

    except sqlite3.Error as e:
        print("Error while inserting data:", e)

def main():
    create_database()

    # Sample data to insert (replace this with actual data and image paths)
    data = [
        {
            "email": "toluwaniojof@gmail.com",
            "name": "Ojo Toluwani",
            "age": 20,
            "occupation": "Engineer",
            "image_path": "Known_Faces/Tolu2.jpg"
        },
        {
            "email": "john.doe@yahoo.com",
            "name": "Balogs Olamlams",
            "age": 25,
            "occupation": "Software Engineer",
            "image_path": "Known_Faces/Ola.jpg"
        },
        {
            "email": "adisadavid@gmail.com",
            "name": "Adisa David",
            "age": 6,
            "occupation": "Data Analyst",
            "image_path": "Known_Faces/David.jpg"
        },
        {
            "email": "somthingif@gmail.com",
            "name": "Something Oluwaife",
            "age": 15,
            "occupation": "Programmer",
            "image_path": "Known_Faces/Ife.jpg"
        },
        {
            "email": "aalimah@gmail.com",
            "name": "Aalimah",
            "age": 10,
            "occupation": "Senior Prefect",
            "image_path": "Known_Faces/Aalimah.jpg"
        }
    ]

    for item in data:
        email = item["email"]
        name = item["name"]
        age = item["age"]
        occupation = item["occupation"]
        image_path = item["image_path"]

        face_encoding = encode_image(image_path)
        if face_encoding is not None:
            insert_data(email, name, age, occupation, str(face_encoding.tolist()), image_path)
        else:
            print(f"Error: No face found in the image: {image_path}")

if __name__ == "__main__":
  main()

Data inserted successfully!
Data inserted successfully!
Data inserted successfully!
Data inserted successfully!
Data inserted successfully!


## Test Face Recognition and DB info retrieval

In [2]:
import cv2
import face_recognition
import sqlite3

DATABASE_NAME = "face_data.db"

def load_known_face_encodings():
    known_face_encodings = {}
    try:
        # Connect to the database
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Load face encodings and other information from the database
        cursor.execute('''
            SELECT name, face_encoding FROM face_data
        ''')

        rows = cursor.fetchall()
        for name, face_encoding in rows:
            known_face_encodings[name] = eval(face_encoding)

        # Close the connection
        conn.close()

    except sqlite3.Error as e:
        print("Error while loading known face encodings:", e)

    return known_face_encodings

def recognize_person(face_encoding, known_face_encodings):
    # Compare the face encoding with known faces
    for name, known_face_encoding in known_face_encodings.items():
        matches = face_recognition.compare_faces([known_face_encoding], face_encoding)

        if any(matches):
            return name

    return None

def main():
    known_face_encodings = load_known_face_encodings()

    # Load Haar Cascade Classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Start webcam capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame.")
            break

        # Convert the frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face_encoding = face_recognition.face_encodings(frame, [(y, x + w, y + h, x)])[0]

            name = recognize_person(face_encoding, known_face_encodings)
            if name:
                # Calculate similarity percentage
                known_face_encoding_arr = [known_face_encodings[name]]
                similarity_scores = face_recognition.face_distance(known_face_encoding_arr, face_encoding)
                similarity_percentage = (1 - similarity_scores[0]) * 100
                if similarity_percentage > 66:
                    color = (90, 255, 0)
                    
                    try:
                        # Connect to the database
                        conn = sqlite3.connect(DATABASE_NAME)
                        cursor = conn.cursor()

                        # Fetch additional information from the database
                        cursor.execute('''
                            SELECT age, occupation, email FROM face_data WHERE name=?
                        ''', (name,))
                        row = cursor.fetchone()

                        # Close the connection
                        conn.close()

                        if row:
                            age, occupation, email = row
                            # Draw a rectangle around the face
                            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                            # text = f"{name}\nAge: {age}\nOccupation: {occupation}\nemail: {email}"
                            cv2.putText(frame, name, (x, y - 140), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                            cv2.putText(frame, str(age), (x, y -110), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                            cv2.putText(frame, str(occupation), (x, y - 80), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                            cv2.putText(frame, str(email), (x, y -50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                            cv2.putText(frame, str(round(similarity_percentage, 2)), (x, y -20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

                    except sqlite3.Error as e:
                        print("Error while fetching additional information:", e)
                else:
                    color = (0, 0, 255)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    cv2.putText(frame, "Unknown Face", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                    cv2.putText(frame, str(similarity_percentage), (x, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                        

        # Show the frame
        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


## Renaming image files in Raw Images folder

In [3]:
import os, tensorflow as tf

count = 0
raw_image_path = "Raw Images"
subs = [f for f in os.listdir(raw_image_path)]

for sub in subs:
  path = os.path.join(raw_image_path, sub)
  
  images = [f for f in os.listdir(path)]
  print(images)
  
  for index, image in enumerate(images, start= 1):
    byte_img = tf.io.read_file(image)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(image, (250,250))
    img = img / 255.0
    # new_name = f"{sub}_{index}.jpg"
    # old_path = os.path.join(path, image)
    # new_path = os.path.join(path, new_name)
    # os.rename(old_path, new_path)
    # print(f"{image} renamed to {new_name}")

['David_1.jpg', 'David_10.jpg', 'David_11.jpg', 'David_12.jpg', 'David_13.jpg', 'David_14.jpg', 'David_15.jpg', 'David_16.jpg', 'David_17.jpg', 'David_18.jpg', 'David_19.jpg', 'David_2.jpg', 'David_20.jpg', 'David_21.jpg', 'David_22.jpg', 'David_23.jpg', 'David_24.jpg', 'David_25.jpg', 'David_26.jpg', 'David_27.jpg', 'David_28.jpg', 'David_29.jpg', 'David_3.jpg', 'David_30.jpg', 'David_31.jpg', 'David_32.jpg', 'David_33.jpg', 'David_34.jpg', 'David_35.jpg', 'David_36.jpg', 'David_37.jpg', 'David_38.jpg', 'David_39.jpg', 'David_4.jpg', 'David_40.jpg', 'David_41.jpg', 'David_42.jpg', 'David_43.jpg', 'David_44.jpg', 'David_45.jpg', 'David_46.jpg', 'David_47.jpg', 'David_48.jpg', 'David_49.jpg', 'David_5.jpg', 'David_50.jpg', 'David_51.jpg', 'David_52.jpg', 'David_53.jpg', 'David_54.jpg', 'David_55.jpg', 'David_56.jpg', 'David_57.jpg', 'David_58.jpg', 'David_59.jpg', 'David_6.jpg', 'David_60.jpg', 'David_61.jpg', 'David_62.jpg', 'David_63.jpg', 'David_64.jpg', 'David_65.jpg', 'David_66.jpg

NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: David_1.jpg : The system cannot find the file specified.
; No such file or directory [Op:ReadFile]

In [5]:
import os

count = 0

# raw_image_path = "Raw Images"

# for name in os.listdir(raw_image_path)

def rename_images_in_subfolders(root_folder):
    # Get a list of sub-folders in the root folder
    subfolders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

    # Loop through each sub-folder
    for subfolder in subfolders:
        subfolder_path = os.path.join(root_folder, subfolder)

        # Check if the sub-folder contains image files
        image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
        if not image_files:
            continue

        # Loop through the image files and rename them with integers
        for index, image_file in enumerate(image_files, start=1):
            new_name = f"{subfolder}_{index}.jpg"  # Change the file extension if needed
            old_path = os.path.join(subfolder_path, image_file)
            new_path = os.path.join(subfolder_path, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed {image_file} to {new_name}")

# Example usage: Replace "path_to_root_folder" with the actual path to your root folder containing the sub-folders.
root_folder = "Raw Images"
rename_images_in_subfolders(root_folder)

# print(os.listdir(raw_image_path))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'David'

In [11]:
import tweepy

In [13]:
import face_recognition

# Authenticate with Twitter API
consumer_key = "Dhzcc5ewCrrg1ZYVeCUM8tSD3"
consumer_secret = "uu1ilepwkP1Wve9TlG9BzVJA3rkkaKcgTsk90AmCx8VAeUJyNF"

access_token = "1390026603225305094-HyyOPObjqOJCARhySI9oZoXOXWXZYP"
access_token_secret = "hplNyOZ2we9KrlYryxkH4fipfeRpdHO6cBOFzIyeaTVy6"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Load and encode the input face image
input_face_image = face_recognition.load_image_file("input_face.jpg")
input_face_encoding = face_recognition.face_encodings(input_face_image)[0]

# Search Twitter profiles with profile pictures
best_match_profile = None
best_similarity_score = 0.0

for tweet in tweepy.Cursor(api.search_users, q="filter:has:profile_image").items():
    profile_image_url = tweet.profile_image_url.replace("_normal", "")  # Use original resolution image
    profile_image = face_recognition.load_image_file(profile_image_url)
    profile_face_encoding = face_recognition.face_encodings(profile_image)[0]
    
    similarity_score = face_recognition.face_distance([input_face_encoding], profile_face_encoding)[0]
    
    if similarity_score > best_similarity_score:
        best_similarity_score = similarity_score
        best_match_profile = tweet

# Display the best match profile information
if best_match_profile:
    print("Best Match Profile:")
    print("Twitter Username:", best_match_profile.screen_name)
    print("Profile Image URL:", best_match_profile.profile_image_url)
    print("Similarity Score:", best_similarity_score)
else:
    print("No matching profile found.")

FileNotFoundError: [Errno 2] No such file or directory: 'input_face.jpg'